In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from sklearn import decomposition, preprocessing
from scipy.cluster.hierarchy import dendrogram

population = pd.read_csv("population_2016_2017.csv")
dispAlim = pd.read_csv("disp_alim_totale_2017.csv")
dispAlimAnimaux = pd.read_csv("disp_alim_animaux_2017.csv")

In [3]:
# Création de dataPop pour la population de chaque pays en 2016 et 2017 :

dataPop = population.loc[ : ,('Code zone','Zone','Année','Valeur')]\
                    .pivot_table(index = ['Code zone','Zone'], columns = 'Année', values='Valeur', aggfunc = sum)\
                    .reset_index()

# Vu dans projet 3 :
# La Chine apparaît deux fois :
# - Chine, continentale (Code zone = 41), donnée officielle
# - Chine (Code zone = 351), agrégat incluant des données non officielles

# Suppression de l'élément "Chine" avec le code zone : 

dataPop = dataPop[dataPop["Code zone"]!=351]

# Création colonne "Différence de population entre 2016 et 2017" :

dataPop['Différence de population entre 2016 et 2017 en %'] = ((dataPop[2017]-dataPop[2016])/dataPop[2016])*100

# Création de dataAlim pour la disponibilité alimentaire totale :

dataAlim = dispAlim.pivot_table(index =['Zone','Code zone'],
                            columns='Élément', values='Valeur', aggfunc = sum)\
                   .loc[ : ,('Disponibilité alimentaire (Kcal/personne/jour)',
                             'Disponibilité de protéines en quantité (g/personne/jour)')]\
                   .reset_index()\
                   .rename(columns={'Disponibilité alimentaire (Kcal/personne/jour)':'Disponibilité alimentaire totale (Kcal/personne)',
                                    'Disponibilité de protéines en quantité (g/personne/jour)':'Disponibilité totale de protéines en quantité (g/personne)'})
dataAlim['Disponibilité alimentaire totale (Kcal/personne)'] *= 365
dataAlim['Disponibilité totale de protéines en quantité (g/personne)'] *= 365

# Suppression des valeurs incohérentes :

dataAlim = dataAlim.drop(dataAlim[dataAlim['Disponibilité alimentaire totale (Kcal/personne)']<0].index)\
                   .drop(dataAlim[dataAlim['Disponibilité totale de protéines en quantité (g/personne)']<0].index)\
                   .drop_duplicates()

# Création de dataAnim pour la disponibilité alimentaire animale :

dataAnim = dispAlimAnimaux.pivot_table(index =['Zone','Code zone'],
                            columns='Élément', values='Valeur', aggfunc = sum)\
                          .loc[ : ,('Disponibilité de protéines en quantité (g/personne/jour)')]\
                          .reset_index()\
                          .rename(columns={'Disponibilité de protéines en quantité (g/personne/jour)':"Disponibilité d'origine animale de protéines en quantité (g/personne)"})
dataAnim["Disponibilité d'origine animale de protéines en quantité (g/personne)"] *= 365

# Suppression des valeurs incohérentes :

dataAnim = dataAnim.drop(dataAnim[dataAnim["Disponibilité d'origine animale de protéines en quantité (g/personne)"]<0].index)\
                   .drop_duplicates()

# Jointure de dataPop, dataAlim et dataAnim : 

data = pd.merge(dataPop, dataAlim)
data = pd.merge(data, dataAnim)

data["Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %"] = (data["Disponibilité d'origine animale de protéines en quantité (g/personne)"]/data['Disponibilité totale de protéines en quantité (g/personne)'])*100

data = data.drop(columns=['Code zone',2016,2017,"Disponibilité d'origine animale de protéines en quantité (g/personne)"])\
           .set_index('Zone')

data

,Différence de population entre 2016 et 2017 en %,Disponibilité alimentaire totale (Kcal/personne),Disponibilité totale de protéines en quantité (g/personne),Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %
Zone,,,,
Arménie,0.294604,1123470.0,35540.05,45.773852
Afghanistan,2.580559,730000.0,19742.85,19.504529
Albanie,-0.078644,1241000.0,43635.75,55.491426
Algérie,2.066020,1222385.0,33912.15,27.671941
Angola,3.377933,828550.0,19750.15,30.456478
...,...,...,...,...
Belgique,0.575371,1375320.0,37065.75,56.986706
Luxembourg,2.183821,1217275.0,38711.90,63.539506
Serbie,-0.274792,1023825.0,30112.50,48.509091


In [4]:
ClassementDemo = data.sort_values(by='Différence de population entre 2016 et 2017 en %')

ClassementDemo.tail(60)

,Différence de population entre 2016 et 2017 en %,Disponibilité alimentaire totale (Kcal/personne),Disponibilité totale de protéines en quantité (g/personne),Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %
Zone,,,,
Turquie,1.614198,1292100.0,36948.95,35.335375
Israël,1.663218,1280785.0,44785.50,59.127954
Pérou,1.675837,1010685.0,28623.30,37.643458
Turkménistan,1.683041,1047915.0,33890.25,43.112547
Honduras,1.706541,931480.0,23418.40,34.756858
Panama,1.726248,1036235.0,29010.20,53.095118
Équateur,1.784233,943890.0,24009.70,45.515354
Mongolie,1.879033,865415.0,30072.35,68.139337
Namibie,1.890977,887315.0,22692.05,36.625382


In [5]:
ClassementDispoAlim = data.sort_values(by='Disponibilité alimentaire totale (Kcal/personne)')

ClassementDispoAlim.head(60)

,Différence de population entre 2016 et 2017 en %,Disponibilité alimentaire totale (Kcal/personne),Disponibilité totale de protéines en quantité (g/personne),Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %
Zone,,,,
République centrafricaine,1.285456,641670.0,16950.60,43.712317
Madagascar,2.716036,694595.0,15673.10,21.425245
Afghanistan,2.580559,730000.0,19742.85,19.504529
Zambie,2.995332,734745.0,21567.85,19.783381
République populaire démocratique de Corée,0.482700,741680.0,19071.25,20.172249
Bahamas,1.010769,745695.0,22436.55,68.342281
Yémen,2.453640,752995.0,19428.95,20.026301
Tchad,3.125262,762850.0,25736.15,30.875053
Tadjikistan,2.501160,767960.0,20217.35,24.282361


In [6]:
ClassementDispoProt = data.sort_values(by="Disponibilité totale de protéines en quantité (g/personne)")

ClassementDispoProt.head(60)

,Différence de population entre 2016 et 2017 en %,Disponibilité alimentaire totale (Kcal/personne),Disponibilité totale de protéines en quantité (g/personne),Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %
Zone,,,,
Libéria,2.516793,778180.0,14559.85,24.517423
Madagascar,2.716036,694595.0,15673.10,21.425245
Guinée-Bissau,2.563901,822345.0,16114.75,21.132503
République centrafricaine,1.285456,641670.0,16950.60,43.712317
Haïti,1.313657,789860.0,18228.10,19.983981
Zimbabwe,1.470101,793145.0,18691.65,22.690881
Mozambique,2.943161,840960.0,18691.65,17.555165
Sao Tomé-et-Principe,1.899326,882570.0,18998.25,30.413064
République populaire démocratique de Corée,0.482700,741680.0,19071.25,20.172249


In [7]:
ClassementPropProt = data.sort_values(by="Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %")

ClassementPropProt.head(60)

,Différence de population entre 2016 et 2017 en %,Disponibilité alimentaire totale (Kcal/personne),Disponibilité totale de protéines en quantité (g/personne),Proportion de protéines d'origine animale par rapport à quantité totale de protéines dans disponibilté alimentaire en %
Zone,,,,
Éthiopie,2.699196,840960.0,23962.25,8.910891
Nigéria,2.641963,899360.0,20410.80,12.750358
Burkina Faso,2.932851,992800.0,27787.45,14.370156
Rwanda,2.674904,808475.0,21622.60,14.770425
Niger,3.913550,941335.0,29889.85,15.239956
République-Unie de Tanzanie,3.037009,874540.0,21819.70,15.623954
Togo,2.510270,886585.0,20921.80,16.050244
Cameroun,2.672847,968345.0,25349.25,16.213103
Malawi,2.702313,966155.0,24827.30,16.847986


In [39]:
PIBparHab = pd.read_csv("API_NY.GDP.PCAP.KD_DS2_fr_csv_v2_1254004.csv")

In [40]:
countryCode = pd.read_csv("CountryCode.csv")

In [41]:
PIBparHab

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"""Aruba"",""ABW"",""PIB par habitant ($ US constant...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Afghanistan,""AFG"",""PIB par habitant ($ US cons...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Angola,""AGO"",""PIB par habitant ($ US constants...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Albanie,""ALB"",""PIB par habitant ($ US constant...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Andorre,""AND"",""PIB par habitant ($ US constant...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,"Kosovo,""XKX"",""PIB par habitant ($ US constants...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
260,"Yémen, Rép. du,""YEM"",""PIB par habitant ($ US c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
261,"Afrique du Sud,""ZAF"",""PIB par habitant ($ US c...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Zambie,""ZMB"",""PIB par habitant ($ US constants...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
countryCode

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,"Afghanistan,AF,AFG,004,ISO 3166-2:AF,Asia,Sout...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Albania,AL,ALB,008,ISO 3166-2:AL,Europe,Southe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Algeria,DZ,DZA,012,ISO 3166-2:DZ,Africa,Northe...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"American Samoa,AS,ASM,016,ISO 3166-2:AS,Oceani...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
244,"Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oce...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
245,"Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
246,"Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western As...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
247,Zambia,ZM,ZMB,894.0,ISO 3166-2:ZM,Africa,Sub-Saharan Africa,Eastern Africa,2.0,202.0,14.0
